# Hyper parameters optimization #

## Parameters cell ##

Parameters are overiddent by papermill when run inside DVC stages



In [1]:
n_folds = 5 # this sets global setting of which how many bootstraps to use
n_hold_out = 1
repeats = 5
n_trials = 10
importance_type = "split"
#first round of optimization

life_history = ["lifespan", "mass_kg", "mtGC", "metabolic_rate", "temperature", "gestation_days"]
trait = "mass_kg"#"lifespan"

debug_local = True #to use local version

In [2]:
from pathlib import Path
import sys
import inspect

local = (Path("..") / "yspecies").resolve()
if debug_local and local.exists():
  sys.path.insert(0, Path("..").as_posix())
  #sys.path.insert(0, local.as_posix())
  print("extending pathes with local yspecies")
  print(sys.path)
  %load_ext autoreload
  %autoreload 2

extending pathes with local yspecies
['..', '/data/sources/yspecies/notebooks', '/opt/miniconda3/envs/yspecies/lib/python38.zip', '/opt/miniconda3/envs/yspecies/lib/python3.8', '/opt/miniconda3/envs/yspecies/lib/python3.8/lib-dynload', '', '/opt/miniconda3/envs/yspecies/lib/python3.8/site-packages', '/opt/miniconda3/envs/yspecies/lib/python3.8/site-packages/IPython/extensions', '/home/antonkulaga/.ipython']


In [3]:
from dataclasses import dataclass, replace
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from typing import *
from yspecies.dataset import *
from yspecies.utils import *
from yspecies.workflow import TupleWith, Repeat, Collect
from yspecies.config import *
from yspecies.preprocess import FeatureSelection, DataExtractor
from yspecies.partition import DataPartitioner, PartitionParameters
from yspecies.selection import ShapSelector
from yspecies.tuning import Tune, MultiObjectiveResults
from yspecies.models import ResultsCV, CrossValidator
from yspecies.results import FeatureSummary
import optuna
from optuna import Study, Trial

In [5]:
#settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import pprint
pp = pprint.PrettyPrinter(indent=4)

#charts settings
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
%matplotlib auto
plt.ioff()
set_matplotlib_formats('svg')

Using matplotlib backend: TkAgg


### Loading data ###
Let's load data from species/genes/expressions selected by select_samples.py notebook

In [6]:
from pathlib import Path
locations: Locations = Locations("./") if Path("./data").exists() else Locations("../")

In [7]:
import logging
log = logging.getLogger('server_logger')
log.setLevel(logging.DEBUG)
fh = logging.FileHandler(locations.logs / "optimization.log")

In [8]:
from loguru import logger

logger.add(locations.logs / "optimization_errors.log", backtrace=True, diagnose=True)
logger.add(locations.logs / "optimization.log", rotation="12:00")     # New file is created each day at noon

2

In [10]:
selection = FeatureSelection(
    samples = ["tissue","species"], #samples metadata to include
    species =  [], #species metadata other then Y label to include
    exclude_from_training = ["species"],  #exclude some fields from LightGBM training
    to_predict = trait, #column to predict
    categorical = ["tissue"],
    select_by = "shap",
    importance_type =  importance_type,
    feature_perturbation = "tree_path_dependent"
)
default_selection

Samples metadata,Species metadata,Genes,Predict label,not_validated species
"['tissue', 'species']",[],all,mass_kg,[]


In [11]:
loader = DataLoader(locations, default_selection)
selections = loader.load_life_history()
selections[trait][0]

expressions,genes,species,samples,Genes Metadata,Species Metadata
"(258, 11982)","(11982, 16)",17,258,"(11982, 2)","(17, 18)"


## Setting up ShapSelector ##

Deciding on selection parameters (which fields to include, exclude, predict)

In [12]:
partition_params = PartitionParameters(n_folds, n_hold_out, 2,   42)

In [15]:
def load_study(trait: str):
    url = f'sqlite:///' +str((locations.interim.optimization / (trait+".sqlite")).absolute())
    print('loading (if exists) study from '+url)
    storage = optuna.storages.RDBStorage(
        url=url
        #engine_kwargs={'check_same_thread': False}
    )
    return optuna.multi_objective.study.create_study(directions=['maximize','minimize','maximize'], storage = storage, study_name = f"{trait}_r2_huber_kendall", load_if_exists = True)

study = load_study(trait)
metrics, params = MultiObjectiveResults.from_study(study).best_metrics_params_r2()
metrics, params

loading (if exists) study from sqlite:////data/sources/yspecies/notebooks/../data/interim/optimization/mass_kg.sqlite


create_study is experimental (supported from v1.4.0). The interface can change in the future.
NSGAIIMultiObjectiveSampler is experimental (supported from v1.5.0). The interface can change in the future.
RandomMultiObjectiveSampler is experimental (supported from v1.4.0). The interface can change in the future.
[I 2020-08-26 18:04:14,123] Using an existing study with name 'mass_kg_r2_huber_kendall' instead of creating a new one.
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
FrozenMultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.


((0.8032359733466332, 13.977538295788246, 0.16672372537437377),
 {'bagging_fraction': 0.7535848906828091,
  'boosting_type': 'gbdt',
  'drop_rate': 0.24853979931346007,
  'feature_fraction': 0.46849567660442387,
  'lambda_l1': 0.2692850701010651,
  'lambda_l2': 0.8326997240667217,
  'learning_rate': 0.06489540021953816,
  'max_depth': 8,
  'max_leaves': 21,
  'min_data_in_leaf': 8,
  'objective': 'regression',
  'metrics': ['l1', 'l2', 'huber']})

In [19]:
res = MultiObjectiveResults.from_study(study)
res

MultiObjectiveResults(best_trials=[<optuna.multi_objective.trial.FrozenMultiObjectiveTrial object at 0x7f0d267f2d90>, <optuna.multi_objective.trial.FrozenMultiObjectiveTrial object at 0x7f0d267f30d0>, <optuna.multi_objective.trial.FrozenMultiObjectiveTrial object at 0x7f0d267f31f0>, <optuna.multi_objective.trial.FrozenMultiObjectiveTrial object at 0x7f0d267f33d0>, <optuna.multi_objective.trial.FrozenMultiObjectiveTrial object at 0x7f0d267f3490>], all_trials=[<optuna.multi_objective.trial.FrozenMultiObjectiveTrial object at 0x7f0d268becd0>, <optuna.multi_objective.trial.FrozenMultiObjectiveTrial object at 0x7f0d267f3160>, <optuna.multi_objective.trial.FrozenMultiObjectiveTrial object at 0x7f0d267f3790>, <optuna.multi_objective.trial.FrozenMultiObjectiveTrial object at 0x7f0d2685e7c0>, <optuna.multi_objective.trial.FrozenMultiObjectiveTrial object at 0x7f0d267f64c0>, <optuna.multi_objective.trial.FrozenMultiObjectiveTrial object at 0x7f0d267f6520>, <optuna.multi_objective.trial.FrozenMul

In [26]:
def objective_parameters(trial: Trial) -> dict:
    return {
        'objective': 'regression',
        'metric': {'mae', 'mse', 'huber'},
        'verbosity': -1,
        'boosting_type': trial.suggest_categorical('boosting_type', ['dart', 'gbdt']),
        'lambda_l1': trial.suggest_uniform('lambda_l1', 0.01, 3.0),
        'lambda_l2': trial.suggest_uniform('lambda_l2', 0.01, 3.0),
        'max_leaves': trial.suggest_int("max_leaves", 15, 25),
        'max_depth': trial.suggest_int('max_depth', 3, 8),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.3, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.3, 1.0),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.1),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 3, 8),
        'drop_rate': trial.suggest_uniform('drop_rate', 0.1, 0.3),
        "verbose": -1
    }
optimization_parameters = objective_parameters

In [27]:
from yspecies.workflow import SplitReduce

def side(i: int):
    print(i)
    return i

prepare_partition = SplitReduce(
    outputs = DataPartitioner(), 
    split = lambda x: [(x[0], replace(partition_params, seed=side(x[2])))], 
    reduce = lambda x, output: (output[0], x[1]) 
)                               
partition_and_cv = Pipeline(
    [
        ("prepare partition", prepare_partition),
        ("shap_computation", ShapSelector()) #('crossvalidator', CrossValidator())        
    ]
)

partition_and_cv_repeat =  Pipeline([
    ("repeat_cv_pipe", Repeat(partition_and_cv, repeats, lambda x, i: [x[0], x[1], i] )),
    #("collect_mean", Collect(fold=lambda results: np.array([r.last("huber") for r in results]).mean()))
    ("collect_mean", Collect(fold=lambda results: (FeatureSummary(results).metrics_average.R2, FeatureSummary(results).metrics_average.huber, FeatureSummary(results).kendall_tau_abs_mean)))    
    ]
    )

p = Pipeline([
     ('extractor', DataExtractor()),
     ('tune', Tune(partition_and_cv_repeat, study = study, n_trials = n_trials, parameters_space = optimization_parameters))    
])

In [ ]:
results = p.fit_transform(selections[trait])
results

MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
FrozenMultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
FrozenMultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.
FrozenMultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.
MultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
FrozenMultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.


0


2020-08-26 18:06:34.999 | INFO     | yspecies.selection:fit:78 - ===== fitting models with seed 0 =====
2020-08-26 18:06:35.014 | INFO     | yspecies.selection:fit:81 - SEED: 0 | FOLD: 0 | VALIDATION_SPECIES: ['Monodelphis_domestica', 'Aotus_nancymaae']
Early stopping is not available in dart mode


[250]	valid_0's l2: 1163.81	valid_0's l1: 14.7818	valid_0's huber: 12.9362


2020-08-26 18:06:37.913 | INFO     | yspecies.selection:fit:81 - SEED: 0 | FOLD: 1 | VALIDATION_SPECIES: ['Sus_scrofa', 'Oryctolagus_cuniculus']


...

[250]	valid_0's l2: 1290.34	valid_0's l1: 18.274	valid_0's huber: 16.0581


2020-08-26 19:25:12.788 | INFO     | yspecies.selection:fit:81 - SEED: 4 | FOLD: 2 | VALIDATION_SPECIES: ['Homo_sapiens', 'Rattus_norvegicus']


[250]	valid_0's l2: 1194.58	valid_0's l1: 18.9309	valid_0's huber: 16.6551


2020-08-26 19:25:14.936 | INFO     | yspecies.selection:fit:81 - SEED: 4 | FOLD: 3 | VALIDATION_SPECIES: ['Phascolarctos_cinereus', 'Ictidomys_tridecemlineatus']


[250]	valid_0's l2: 1066.6	valid_0's l1: 12.5845	valid_0's huber: 10.9274


In [ ]:
for i, t in enumerate(best):
    trait_path = locations.metrics.optimization / trait
    if not trait_path.exists():
        trait_path.mkdir()
    path = trait_path / f"{str(i)}.json"    
    print(f"writing parameters to {path}")
    with open(path, 'w') as f:
        params = t.params
        values = t.values
        to_write = {"number": t.number,"params": params, "metrics": {"R2":values[0], "huber": values[1], "kendall_tau": values[2]}}
        json.dump(to_write, f, sort_keys=True, indent=4)